In [1]:
import pygame
import random
import copy
import sys

# Sudoku Logic
def is_valid(board, row, col, num):
    for i in range(9):
        if board[row][i] == num or board[i][col] == num:
            return False
    start_row, start_col = 3 * (row // 3), 3 * (col // 3)
    for i in range(3):
        for j in range(3):
            if board[start_row + i][start_col + j] == num:
                return False
    return True

def solve_board(board):
    for row in range(9):
        for col in range(9):
            if board[row][col] == 0:
                for num in range(1, 10):
                    if is_valid(board, row, col, num):
                        board[row][col] = num
                        if solve_board(board):
                            return True
                        board[row][col] = 0
                return False
    return True

def generate_full_board():
    board = [[0]*9 for _ in range(9)]
    solve_board(board)
    return board

def generate_puzzle(board, holes):
    puzzle = copy.deepcopy(board)
    count = 0
    while count < holes:
        row = random.randint(0, 8)
        col = random.randint(0, 8)
        if puzzle[row][col] != 0:
            puzzle[row][col] = 0
            count += 1
    return puzzle

def is_complete(board):
    for row in range(9):
        for col in range(9):
            if board[row][col] == 0 or not is_valid(board, row, col, board[row][col]):
                return False
    return True

#  Pygame Setup 
pygame.init()
WIDTH, HEIGHT = 600, 700
WIN = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Sudoku")

# Fonts
FONT = pygame.font.SysFont("segoeui", 40)
SMALL_FONT = pygame.font.SysFont("segoeui", 18)
BIG_FONT = pygame.font.SysFont("segoeui", 50)
PENCIL_FONT = pygame.font.SysFont("segoeui", 15)

# Dark Mode Colors – Aesthetic Theme
WHITE = (240, 240, 240)
BLACK = (20, 20, 20)
GREY = (120, 120, 120)
LIGHT_GREY = (60, 60, 60)
BLUE = (100, 170, 255)
RED = (255, 100, 100)
LIGHTBLUE = (70, 130, 180)
GREEN = (120, 255, 180)
BG_COLOR = (30, 30, 30)
PENCIL_GREY = (180, 180, 180)


DIFFICULTY = 'Medium'

# UI Helpers 
def draw_grid(win):
    gap = WIDTH // 9
    for i in range(10):
        line_width = 2 if i % 3 == 0 else 1
        line_color = WHITE if i % 3 == 0 else GREY
        pygame.draw.line(win, line_color, (0, i * gap), (WIDTH, i * gap), line_width)
        pygame.draw.line(win, line_color, (i * gap, 0), (i * gap, WIDTH), line_width)

def draw_board(win, board, original, selected, chances, pencil_marks, solution):
    win.fill(BG_COLOR)
    gap = WIDTH // 9
    for i in range(9):
        for j in range(9):
            num = board[i][j]
            x, y = j * gap, i * gap
            if num != 0:
                color = GREY if original[i][j] != 0 else (BLUE if num == solution[i][j] else RED)
                text = FONT.render(str(num), True, color)
                win.blit(text, (x+20, y+10))
            elif pencil_marks[i][j]:
                for idx, val in enumerate(pencil_marks[i][j]):
                    if val != 0:
                        px = x + 5 + (idx % 3) * 18
                        py = y + 5 + (idx // 3) * 18
                        mark = PENCIL_FONT.render(str(val), True, GREY)
                        win.blit(mark, (px, py))

    if selected:
        row, col = selected
        pygame.draw.rect(win, LIGHTBLUE, (col*gap+2, row*gap+2, gap-4, gap-4), 0, border_radius=5)

    draw_grid(win)
    instr1 = SMALL_FONT.render("Click cell, type 1-9 | P+1-9 = pencil | H = hint | F = fast pencil", True, BLACK)
    instr2 = SMALL_FONT.render(f"Chances left: {chances}", True, RED if chances < 3 else GREEN)
    win.blit(instr1, (10, 610))
    win.blit(instr2, (10, 640))

    pygame.display.update()

def get_cell_from_pos(pos):
    x, y = pos
    if x < WIDTH and y < WIDTH:
        return y // (WIDTH // 9), x // (WIDTH // 9)
    return None

def show_message(win, msg, color):
    win.fill(BG_COLOR)
    text = BIG_FONT.render(msg, True, color)
    win.blit(text, (WIDTH//2 - text.get_width()//2, HEIGHT//2 - text.get_height()//2))
    pygame.display.update()
    pygame.time.delay(2500)

# Game Logic 
def get_difficulty_holes():
    return {"Easy": 30, "Medium": 45, "Hard": 60}[DIFFICULTY]

def get_possible_numbers(board, row, col):
    return [n for n in range(1, 10) if is_valid(board, row, col, n)]

def auto_fill_pencil(board):
    pencils = [[[] for _ in range(9)] for _ in range(9)]
    for i in range(9):
        for j in range(9):
            if board[i][j] == 0:
                pencils[i][j] = get_possible_numbers(board, i, j)
    return pencils

def give_hint(board, solution, puzzle):
    for i in range(9):
        for j in range(9):
            if board[i][j] == 0:
                board[i][j] = solution[i][j]
                return

#  Main Game Loop
def play_sudoku_gui():
    global DIFFICULTY
    full_board = generate_full_board()
    puzzle = generate_puzzle(full_board, get_difficulty_holes())
    current_board = copy.deepcopy(puzzle)
    solution = copy.deepcopy(full_board)

    selected = None
    chances = 3
    pencil_marks = [[[] for _ in range(9)] for _ in range(9)]
    penciling = False

    while True:
        draw_board(WIN, current_board, puzzle, selected, chances, pencil_marks, solution)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit(); sys.exit()
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    return
                if event.key == pygame.K_p:
                    penciling = True
                    continue
                if event.key == pygame.K_h:
                    give_hint(current_board, solution, puzzle)
                if event.key == pygame.K_f:
                    pencil_marks = auto_fill_pencil(current_board)

                if selected and event.unicode in '123456789':
                    r, c = selected
                    if puzzle[r][c] == 0:
                        val = int(event.unicode)
                        if penciling:
                            if val in pencil_marks[r][c]: pencil_marks[r][c].remove(val)
                            else: pencil_marks[r][c].append(val)
                            penciling = False
                        else:
                            prev_val = current_board[r][c]
                            current_board[r][c] = val
                            pencil_marks[r][c] = []
                            if val == solution[r][c]:
                                if is_complete(current_board):
                                    draw_board(WIN, current_board, puzzle, None, chances, pencil_marks, solution)
                                    show_message(WIN, "Congratulations! You have cleared the level", GREEN)
                                    return
                            else:
                                if prev_val != solution[r][c]:
                                    chances -= 1
                                    if chances == 0:
                                        show_message(WIN, "Game Over!", RED)
                                        return

            if event.type == pygame.MOUSEBUTTONDOWN:
                selected = get_cell_from_pos(pygame.mouse.get_pos())

# Menu + Rules 
def show_rules():
    win.fill(BG_COLOR)
    rules = [
        "Sudoku Rules:",
        "- Fill the 9x9 grid with numbers 1–9.",
        "- Each row, column, and 3x3 grid must have all digits once.",
        "- You have 3 chances to make mistakes.",
        "- Use P+number to pencil possible values.",
        "- Press H for a hint and F for fast pencil (autofill pencils).",
        "",
        "Click anywhere or press ESC to return."
    ]
    for i, line in enumerate(rules):
        text = SMALL_FONT.render(line, True, BLACK)
        WIN.blit(text, (30, 30 + i * 30))
    pygame.display.update()

    while True:
        for e in pygame.event.get():
            if e.type == pygame.QUIT:
                pygame.quit(); sys.exit()
            if e.type in [pygame.MOUSEBUTTONDOWN, pygame.KEYDOWN]:
                return

def show_menu():
    global DIFFICULTY
    while True:
        WIN.fill(BG_COLOR)
        title = BIG_FONT.render("Sudoku Game", True, WHITE)
        play_btn = FONT.render("Play", True, BLUE)
        rules_btn = FONT.render("Rules", True, WHITE)
        diff_label = SMALL_FONT.render(f"Difficulty: {DIFFICULTY} (Click to cycle)", True, GREY)

        WIN.blit(title, (WIDTH//2 - title.get_width()//2, 80))
        WIN.blit(play_btn, (WIDTH//2 - play_btn.get_width()//2, 200))
        WIN.blit(rules_btn, (WIDTH//2 - rules_btn.get_width()//2, 270))
        WIN.blit(diff_label, (WIDTH//2 - diff_label.get_width()//2, 350))

        pygame.display.update()

        for e in pygame.event.get():
            if e.type == pygame.QUIT:
                pygame.quit(); sys.exit()
            if e.type == pygame.MOUSEBUTTONDOWN:
                x, y = pygame.mouse.get_pos()
                if 200 <= y <= 240:
                    play_sudoku_gui()
                elif 270 <= y <= 310:
                    show_rules()
                elif 350 <= y <= 380:
                    DIFFICULTY = {"Easy": "Medium", "Medium": "Hard", "Hard": "Easy"}[DIFFICULTY]

# Start Game 
show_menu()


pygame 2.6.1 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

C:\ProgramData\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
